## Ceate Dataset

In [1]:
import torch
from torch.utils.data import Dataset

import numpy as np
import csv


class StockDataset(Dataset):
    ''' Dataset for loading and preprocessing the dataset '''
    def __init__(self,
                 path,
                 mode='train'
              ):
        self.mode = mode

        # Read data into numpy arrays
        with open(path, 'r') as fp:
          data = list(csv.reader(fp))
          data = np.array(data[1:])[:, 4:].astype(float)
        
        target = data[:, -1]
        data = data[:, list(range(22))]

        if mode == 'test':
            # Testing data
            self.data = torch.FloatTensor(data)
            self.target = torch.LongTensor(target)
        else:
            # Training data (train/dev sets)
        
            # Splitting training data into train & dev sets
            gap = int(len(data) * 0.8)
            if mode == 'train':
                
                self.data = torch.FloatTensor(data[:gap])
                self.target = torch.LongTensor(target[:gap])
            elif mode == 'dev':
                            
            # Convert data into PyTorch tensors
                self.data = torch.FloatTensor(data[gap:])
                self.target = torch.LongTensor(target[gap:])

        # Normalize features
        self.data = \
            (self.data - self.data.mean(dim=0, keepdim=True)) \
            / self.data.std(dim=0, keepdim=True)

        self.dim = self.data.shape[1]

        # print(f'Finished reading the {mode} set of Dataset ({len(self.data)} samples found, each dim = {self.dim})')

    def __getitem__(self, index):

        return self.data[index], self.target[index]

    def __len__(self):

        return len(self.data)


## Create Model

In [2]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.layer1 = nn.Linear(22, 512)
        self.layer1_bn=nn.BatchNorm1d(512)

        self.layer2 = nn.Linear(22, 256)
        self.layer2_bn=nn.BatchNorm1d(256)

        self.layer3 = nn.Linear(256, 128)
        self.layer3_bn=nn.BatchNorm1d(128)

        self.layer4 = nn.Linear(128, 64)
        self.layer4_bn=nn.BatchNorm1d(64)

        self.layer5 = nn.Linear(64, 32)
        self.layer5_bn=nn.BatchNorm1d(32)

        self.drop = nn.Dropout(0.5)

        self.out = nn.Linear(32, 3) 
        
        self.act_fn = nn.ReLU()
        

    def forward(self, x):
        # x = self.drop(x)

        # x = self.layer1(x)
        # x = self.layer1_bn(x)
        # x = self.act_fn(x)
        # x = self.drop(x)

        x = self.layer2(x)
        x = self.layer2_bn(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer3(x)
        x = self.layer3_bn(x)
        x = self.act_fn(x)
        x = self.drop(x)
 
        x = self.layer4(x)
        x = self.layer4_bn(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.layer5(x)
        x = self.layer5_bn(x)
        x = self.act_fn(x)
        x = self.drop(x)

        x = self.out(x)

        
        return x

In [3]:

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score 
plt.ioff()
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# fix random seed for reproducibility
same_seeds(0)

# get device 
device = 'cpu'
print(f'DEVICE: {device}')

# training parameters
num_epoch = 200        # number of training epoch
BATCH_SIZE = 32
learning_rate = 0.001       # learning rate


def macro_precision(y_true, y_predict):
    hit_count = {0: 0, 1: 0, 2: 0}
    loss_count = 0
    for i, prediction in enumerate(y_predict):
        if y_true[i] == prediction:
            hit_count[prediction] += 1
        elif (y_true[i] == 0 and prediction == 2) or (y_true[i] == 2 and prediction == 0):
            loss_count += 1
    # r1 = hit_count[0]/y_true.count(0)
    # r2 = hit_count[1]/y_true.count(1)
    # r3 = hit_count[2]/y_true.count(2)

    if hit_count[0] == 0:
        p1 = 0
    else:
        p1 = hit_count[0]/y_predict.count(0)
    if hit_count[1] == 0:
        p2 = 0
    else:
        p2 = hit_count[1]/y_predict.count(1)
    if hit_count[2] == 0:
        p3 = 0
    else:
        p3 = hit_count[2]/y_predict.count(2)

    # f1 = 2 * p1 * r1 / (p1 + r1)
    # f2 = 2 * p2 * r2 / (p2 + r2)
    # f3 = 2 * p3 * r3 / (p3 + r3)
    
    # return (p1 + p2 + p3) / 3
    return hit_count[0] + hit_count[2] - loss_count

def plot_learning_curve(acc_record, title, y_label, i , d, limit_y = True):
    ''' Plot learning curve of your DNN (train & dev loss) '''
    total_epoch = len(acc_record['train'])
    x = range(total_epoch)
    # x_2 = x_1[::len(acc_record['train']) // len(acc_record['dev'])]
    figure(figsize=(6, 4))
    plt.plot(x, acc_record['train'], c='tab:red', label='train')
    plt.plot(x, acc_record['dev'], c='tab:cyan', label='dev')
    if limit_y:
        plt.ylim(0.0, 1.0)
    plt.xlabel('Training Epoch')
    plt.ylabel(y_label)
    plt.title('Learning curve of {}'.format(title))
    plt.legend()
    plt.savefig(f'./models/fig/{y_label}_{i}_{d}d.png')
    plt.close()

DEVICE: cpu


## Training

In [4]:
error_list = []

for i in range(1, 201):
    print(f'Model {i}')
    for d in [10, 20, 60, 120]:
        
# for error in redo_list:
#         i, d = error
#         i = int(i)
#         d = int(d[:-1])
#         print(f'Model {i}_{d}d')
        early_stop_count = 0
        try:
            train_set = StockDataset(f'./data/training_data/training_data_{i}_{d}d.csv', mode = 'test')
            val_set = StockDataset(f'./data/training_data/testing_data_{i}_{d}d.csv', mode = 'test')
            train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True) #only shuffle the training data
            val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

            # the path where checkpoint saved
            model_path = f'./models/models/model_{i}_{d}d.ckpt'

            # create model, define a loss function, and optimizer
            model = Classifier().to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

            # Training

            best_acc = 0.0
            best_macro_precision = 0.0

            acc_record = {'train': [], 'dev': []}
            macro_f1_record = {'train': [], 'dev': []}
            macro_precision_score_record = {'train': [], 'dev': []}

            # init accuracy
            train_acc = 0.0
            val_acc = 0.0

            train_labels = []
            train_predictions = []
            val_labels = []
            val_predictions = []
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, val_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    val_acc += (val_pred.cpu() == labels.cpu()).sum().item() 
                    for y in val_pred.cpu().numpy():
                        val_predictions.append(y)

                    for y in labels.cpu().numpy():
                        val_labels.append(y)
                
                for data in train_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs) 
                    _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    train_acc += (train_pred.cpu() == labels.cpu()).sum().item() 
                    for y in train_pred.cpu().numpy():
                        train_predictions.append(y)

                    for y in labels.cpu().numpy():
                        train_labels.append(y)

                acc_record['dev'].append(val_acc/len(val_set))
                acc_record['train'].append(train_acc/len(train_set))
                macro_f1_record['dev'].append(f1_score(val_labels, val_predictions, average='macro'))
                macro_f1_record['train'].append(f1_score(train_labels, train_predictions, average='macro'))
                macro_precision_score_record['dev'].append(macro_precision(val_labels, val_predictions))
                macro_precision_score_record['train'].append(macro_precision(train_labels, train_predictions))
                torch.save(model.state_dict(), model_path)

            # start training
            for epoch in range(num_epoch):
            
                train_acc = 0.0
                train_loss = 0.0
                val_acc = 0.0
                val_loss = 0.0

                train_labels = []
                train_predictions = []
                val_labels = []
                val_predictions = []

                # training
                model.train() # set the model to training mode
                for data in train_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad() 
                    outputs = model(inputs) 
                    batch_loss = criterion(outputs, labels)
                    _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    batch_loss.backward() # compute gradient
                    optimizer.step() # update model with optimizer

                    train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
                    train_loss += batch_loss.item()
                    for y in train_pred.cpu().numpy():
                        train_predictions.append(y)

                    for y in labels.cpu().numpy():
                        train_labels.append(y)

                
                acc_record['train'].append(train_acc/len(train_set))

                train_f1 = f1_score(train_labels, train_predictions, average='macro')
                macro_f1_record['train'].append(train_f1)

                train_weighted_precision = macro_precision(train_labels, train_predictions)
                macro_precision_score_record['train'].append(train_weighted_precision)

                # validation

                model.eval() # set the model to evaluation mode
                with torch.no_grad():
                    for data in val_loader:
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        batch_loss = criterion(outputs, labels) 
                        _, val_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
                    
                        val_acc += (val_pred.cpu() == labels.cpu()).sum().item() 
                        val_loss += batch_loss.item()
                        for y in val_pred.cpu().numpy():
                            val_predictions.append(y)

                        for y in labels.cpu().numpy():
                            val_labels.append(y)


                    acc_record['dev'].append(val_acc/len(val_set))

                    val_f1 = f1_score(val_labels, val_predictions, average='macro')
                    macro_f1_record['dev'].append(val_f1)

                    val_weighted_precision = macro_precision(val_labels, val_predictions)
                    macro_precision_score_record['dev'].append(val_weighted_precision)


                    # if the model improves, save a checkpoint at this epoch
                    if val_weighted_precision > best_macro_precision:
                        best_macro_precision = val_weighted_precision
                        torch.save(model.state_dict(), model_path)
                        # print('[{:03d}/{:03d}] Train Acc: {:3.6f} F1: {:.3f} wP: {:.3f} Loss: {:3.6f} | \
                        # Val Acc: {:3.6f} F1: {:.3f} wP: {:.3f} loss: {:3.6f}'
                            # .format(epoch + 1, num_epoch, train_acc/len(train_set), train_f1, train_weighted_precision, train_loss/len(train_loader), 
                            #                             val_acc/len(val_set), val_f1, val_weighted_precision, val_loss/len(val_loader)))
                        # print('saving model with Val Acc {:.3f}'.format(best_acc/len(val_set)))
                        # print('saving model with Val Precision {:.3f}'.format(best_macro_precision))
                        early_stop_count = 0
                    else:
                        early_stop_count += 1
            
                # if early_stop_count >= 20:
                #     break

            plot_learning_curve(acc_record, 'deep model', 'Accuracy', i, d)
            plot_learning_curve(macro_precision_score_record, 'deep model', 'Macro Precision', i, d, limit_y=False)
            plot_learning_curve(macro_f1_record, 'deep model', 'Macro F1 Score', i, d)
        except Exception as e:
            error_list.append([i, d, str(e)])
            


Model 1
Model 2
Model 3
Model 4
Model 5
Model 6
Model 7
Model 8
Model 9
Model 10
Model 11
Model 12
Model 13
Model 14
Model 15
Model 16
Model 17
Model 18
Model 19
Model 20
Model 21
Model 22
Model 23
Model 24
Model 25
Model 26
Model 27
Model 28
Model 29
Model 30
Model 31
Model 32
Model 33
Model 34
Model 35
Model 36
Model 37
Model 38
Model 39
Model 40
Model 41
Model 42
Model 43
Model 44
Model 45
Model 46
Model 47
Model 48
Model 49
Model 50
Model 51
Model 52
Model 53
Model 54
Model 55
Model 56
Model 57
Model 58
Model 59
Model 60
Model 61
Model 62
Model 63
Model 64
Model 65
Model 66
Model 67
Model 68
Model 69
Model 70
Model 71
Model 72
Model 73
Model 74
Model 75
Model 76
Model 77
Model 78
Model 79
Model 80
Model 81
Model 82
Model 83
Model 84
Model 85
Model 86
Model 87
Model 88
Model 89
Model 90
Model 91
Model 92
Model 93
Model 94
Model 95
Model 96
Model 97
Model 98
Model 99
Model 100
Model 101
Model 102
Model 103
Model 104
Model 105
Model 106
Model 107
Model 108
Model 109
Model 110
Model 11

In [6]:
error_list

[]

In [6]:
import json
    
with open("errors.json", "w") as outfile:
    json.dump(error_list, outfile)
with open('errors.json') as jsonfile:
    redo_list = json.load(jsonfile)    

## Testing

In [7]:

def plot_confusion_matrix(cm,
                          target_names,
                          model,
                          d,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    # plt.xlabel(f'Predicted label\naccuracy={accuracy}')
    plt.xlabel('Predicted label')
    plt.savefig(f'./models/performance/model_{model}_{d}d.png')
    plt.close()

## Average annualized return

In [8]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import csv
import json

with open('performance.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['model', 'd', 'acc', 'r1', 'r2', 'r3', 'p1', 'p2', 'p3', 'f1', 'f2', 'f3'])

with open('trend_params.json') as jsonfile:
    code_para_map = json.load(jsonfile)

with open('codes.txt', 'r') as f:
    codes = f.read().split()

ROI = {10: 0, 20: 0, 60:0, 120:0}

for i in range(1, 201):
  
  print(f'model {i}')
  for d in [10, 20, 60, 120]:
    # print(d)
    model_path = f'./models/models/model_{i}_{d}d.ckpt'
    # create testing dataset
    test_set = StockDataset(f'./data/training_data/testing_data_{i}_{d}d.csv', mode = 'test')
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

    # create model and load weights from checkpoint
    model = Classifier().to(device)
    model.load_state_dict(torch.load(model_path))

    model.eval() # set the model to evaluation mode
    with torch.no_grad():
      test_acc = 0.0
      predict = []
      label = []
      for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs) 
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        test_acc += (test_pred.cpu() == labels.cpu()).sum().item() 

        for y in test_pred.cpu().numpy():
          predict.append(y)

        for y in labels.cpu().numpy():
          label.append(y)

    hit_count = {0: 0, 1: 0, 2: 0}
    loss_count = 0
    for j, prediction in enumerate(predict):
      if label[j] == prediction:
        hit_count[prediction] += 1
      elif (label[j] == 0 and prediction == 2) or (label[j] == 2 and prediction == 0):
        loss_count += 1
   
    acc = (hit_count[0] + hit_count[1] + hit_count[2]) / (label.count(0) + label.count(1) + label.count(2))
    acc = round(acc, 4)
    
    if label.count(0) == 0:
      r1 = 0
    else:
      r1 = round(hit_count[0]/label.count(0), 4)

    if label.count(1) == 0:
      r2 = 0
    else:
      r2 = round(hit_count[1]/label.count(1), 4)

    if label.count(2) == 0:
      r3 = 0
    else:
      r3 = round(hit_count[2]/label.count(2), 4)

    if predict.count(0) == 0:
      p1 = 0
    else:
      p1 = round(hit_count[0]/predict.count(0), 4)

    if predict.count(1) == 0:
      p2 = 0
    else:
      p2 = round(hit_count[1]/predict.count(1), 4)

    if predict.count(2) == 0:
      p3 = 0
    else:
      p3 = round(hit_count[2]/predict.count(2), 4)

    if hit_count[0] == 0:
      f1 = 0
    else:
      f1 = round(2 * p1 * r1 / (p1 + r1), 4)

    if hit_count[1] == 0:
      f2 = 0
    else:
      f2 = round(2 * p2 * r2 / (p2 + r2), 4)

    if hit_count[2] == 0:
      f3 = 0
    else:
      f3 = round(2 * p3 * r3 / (p3 + r3), 4)

    with open('performance.csv', 'a', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow([i, d, acc, r1, r2, r3, p1, p2, p3, f1, f2, f3])

    plot_confusion_matrix(confusion_matrix(label, predict),
                          ['Down', 'Stable' ,'Up'],
                          i,
                          d,
                          normalize    = False
                         )
    fluctuation = code_para_map[codes[i-1]][f'{d}d'][0] - 1
    ROI[d] += fluctuation * (hit_count[0] + hit_count[2] - loss_count) / len(predict)

model 1
model 2
model 3
model 4
model 5
model 6
model 7
model 8
model 9
model 10
model 11
model 12
model 13
model 14
model 15
model 16
model 17
model 18
model 19
model 20
model 21
model 22
model 23
model 24
model 25
model 26
model 27
model 28
model 29
model 30
model 31
model 32
model 33
model 34
model 35
model 36
model 37
model 38
model 39
model 40
model 41
model 42
model 43
model 44
model 45
model 46
model 47
model 48
model 49
model 50
model 51
model 52
model 53
model 54
model 55
model 56
model 57
model 58
model 59
model 60
model 61
model 62
model 63
model 64
model 65
model 66
model 67
model 68
model 69
model 70
model 71
model 72
model 73
model 74
model 75
model 76
model 77
model 78
model 79
model 80
model 81
model 82
model 83
model 84
model 85
model 86
model 87
model 88
model 89
model 90
model 91
model 92
model 93
model 94
model 95
model 96
model 97
model 98
model 99
model 100
model 101
model 102
model 103
model 104
model 105
model 106
model 107
model 108
model 109
model 110
model 11

In [9]:
# Average return
for d in ROI:
    ROI[d] /= 200
ROI

{10: 0.005827673611111111,
 20: 0.012384726562499997,
 60: 0.03377372767857143,
 120: 0.053692687500000016}

In [10]:
# Average annualized return
print('15d', round((((ROI[10]+1)**24)-1)*100, 2))
print('30d', round((((ROI[20]+1)**12)-1)*100, 2))
print('90d', round((((ROI[60]+1)**4)-1)*100, 2))
print('180d', round((((ROI[120]+1)**2)-1)*100, 2))

15d 14.97
30d 15.92
90d 14.21
180d 11.03


In [11]:
# Average annualized return after tax
print('15d', round((((ROI[10]+1-0.0038)**24)-1)*100, 2))
print('30d', round((((ROI[20]+1-0.0038)**12)-1)*100, 2))
print('90d', round((((ROI[60]+1-0.0038)**4)-1)*100, 2))
print('180d', round((((ROI[120]+1-0.0038)**2)-1)*100, 2))

15d 4.98
30d 10.8
90d 12.54
180d 10.23


## Average annualized return of top 20 stocks selection

In [12]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import json

with open('trend_params.json') as jsonfile:
    code_para_map = json.load(jsonfile)

with open('codes.txt', 'r') as f:
    codes = f.read().split()


predicted_label = {10: [[] for _ in range(295)], 20: [[] for _ in range(285)],
                   60: [[] for _ in range(245)], 120:[[] for _ in range(185)]}
                  
predicted_value = {10: [[] for _ in range(295)], 20: [[] for _ in range(285)], 
                   60: [[] for _ in range(245)], 120:[[] for _ in range(185)]}

true_label =      {10: [[] for _ in range(295)], 20: [[] for _ in range(285)], 
                   60: [[] for _ in range(245)], 120:[[] for _ in range(185)]}

for i in range(1, 201):
  print(f'model {i}')
  for d in [10, 20, 60, 120]:
    
    model_path = f'./models/models/model_{i}_{d}d.ckpt'
    # create testing dataset
    test_set = StockDataset(f'./data/training_data/testing_data_{i}_{d}d.csv', mode = 'test')
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

    # create model and load weights from checkpoint
    model = Classifier().to(device)
    model.load_state_dict(torch.load(model_path))

    model.eval() # set the model to evaluation mode
    with torch.no_grad():
      
      days = 0
      for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = nn.Softmax(dim=1)(model(inputs))
        
        test_value, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
              

        for j in range(len(outputs)):
          predicted_value[d][days + j].append(test_value.cpu().numpy()[j])
          predicted_label[d][days + j].append(test_pred.cpu().numpy()[j])
          true_label[d][days + j].append(labels.cpu().numpy()[j])
            
        days += len(outputs)

    # gain = code_para_map[codes[i-1]][f'{d}d'][0] - 1
    
    # ROI[d] += gain * (hit_count[0] + hit_count[2] - loss_count) / len(predict)

model 1
model 2
model 3
model 4
model 5
model 6
model 7
model 8
model 9
model 10
model 11
model 12
model 13
model 14
model 15
model 16
model 17
model 18
model 19
model 20
model 21
model 22
model 23
model 24
model 25
model 26
model 27
model 28
model 29
model 30
model 31
model 32
model 33
model 34
model 35
model 36
model 37
model 38
model 39
model 40
model 41
model 42
model 43
model 44
model 45
model 46
model 47
model 48
model 49
model 50
model 51
model 52
model 53
model 54
model 55
model 56
model 57
model 58
model 59
model 60
model 61
model 62
model 63
model 64
model 65
model 66
model 67
model 68
model 69
model 70
model 71
model 72
model 73
model 74
model 75
model 76
model 77
model 78
model 79
model 80
model 81
model 82
model 83
model 84
model 85
model 86
model 87
model 88
model 89
model 90
model 91
model 92
model 93
model 94
model 95
model 96
model 97
model 98
model 99
model 100
model 101
model 102
model 103
model 104
model 105
model 106
model 107
model 108
model 109
model 110
model 11

In [13]:
tax_rate = 0.0038
annualized_return = {10: [], 20: [], 60: [], 120: []}

for d in [10, 20, 60, 120]:
    for day_shift in range(0, 65):
        compound_interest = 1
        for day in range(0, 240, d):
            predicted_value_n = predicted_value[d][day + day_shift]
            predicted_label_n = predicted_label[d][day + day_shift]
            true_label_n = true_label[d][day + day_shift]

            sorted_value = []
            for i, value in enumerate(predicted_value_n):
                sorted_value.append((value, i))
            sorted_value = sorted(sorted_value)
            
            roi = 0
            stock_selected_count = 0

            for value, idx in reversed(sorted_value):
                if predicted_label_n[idx] in [0, 2]:
                    if predicted_label_n[idx] == true_label_n[idx]:
                        roi += code_para_map[codes[idx]][f'{d}d'][0] - 1
                    elif (predicted_label_n[idx] == 0 and true_label_n[idx] == 2) or \
                        (predicted_label_n[idx] == 2 and true_label_n[idx] == 0):
                        roi -= code_para_map[codes[idx]][f'{d}d'][0] - 1
                    stock_selected_count += 1
                    if stock_selected_count == 20:
                        break
                
            roi = (roi / 20) - tax_rate # roi in the day
            compound_interest *= 1 + roi

        annualized_return[d].append(compound_interest)


In [14]:
from statistics import mean
print(round((mean(annualized_return[10]) - 1) * 100, 2))
print(round((mean(annualized_return[20]) - 1) * 100, 2))
print(round((mean(annualized_return[60]) - 1) * 100, 2))
print(round((mean(annualized_return[120]) - 1) * 100, 2))

15.69
14.09
17.95
10.75


In [15]:
import pandas as pd
from statistics import mean

fluctuation = []
df = pd.read_csv(f'./data/indicator_data/taiex.csv')
df = df[(df['date'] >= '2020-12-31') & (df['date'] <= '2022-04-08')]
for i in range(len(df)-240):
    fluctuation.append(abs((df['TAIEX'].iloc[i + 240] / df['TAIEX'].iloc[i]) - 1))
print(round(mean(fluctuation) * 100, 2))

11.64
